<a href="https://colab.research.google.com/github/2hindas/SVM_a_priori/blob/MNIST-committee/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install python-mnist

In [0]:
import numpy as np
from sklearn import svm
import scipy.ndimage as im
from sklearn.metrics import accuracy_score
from scipy.stats import mode


class EnsembleSVM:

    def __init__(self, train_features, train_targets, test_features, test_targets, constant):
        self.features = train_features
        self.targets = train_targets
        self.test_features = test_features
        self.test_targets = test_targets
        self.num_features = self.features.shape[1]
        self.sqrt_features = int(np.sqrt(self.num_features))
        self.models = []
        self.C = constant
        self.original_model = svm.SVC(kernel='rbf', cache_size=1000, C=constant)
        
        self.original_model.fit(train_features, train_targets)

        print(accuracy_score(test_targets, self.original_model.predict(test_features)))

        self.support_vectors = train_features[self.original_model.support_]
        self.support_vector_targets = train_targets[self.original_model.support_]
        self.feat = self.support_vectors
        self.targ = self.support_vector_targets
        print(f"Support Vectors: {len(self.feat)}")


    def train(self, replace=False):
        model = svm.SVC(kernel='rbf', cache_size=1000, C=self.C)
        print(f"Training set size: {len(self.feat)}")
        model.fit(self.feat, self.targ)
        self.models.append(model)
        if replace:
            self.support_vectors = self.feat[model.support_]
            self.support_vector_targets = self.targ[model.support_]
        self.feat = self.support_vectors
        self.targ = self.support_vector_targets


    def add_translations(self, directions, min_trans, max_trans):
          for d in directions:
              for i in range(min_trans, max_trans + 1):
                  num_sv = len(self.support_vectors)
                  transformation = im.shift(
                      self.support_vectors.reshape(
                          (num_sv, self.sqrt_features, self.sqrt_features)),
                      (0, d[0] * i, d[1] * i), mode='constant', cval=-1)
                  translated_features = transformation.reshape((num_sv, self.num_features))
                  self.targ = np.append(self.targ, self.support_vector_targets)
                  self.feat = np.vstack((translated_features, self.feat))


    def add_rotation(self, min_degrees, max_degrees, step_size):
          for angle in range(min_degrees, max_degrees + 1, step_size):
              if angle == 0:
                  continue
              num_sv = len(self.support_vectors)
              transformation = im.rotate(
                  self.support_vectors.reshape((num_sv, self.sqrt_features, self.sqrt_features)),
                  axes=(1, 2), order=1, angle=angle,
                  mode='constant', cval=-1, reshape=False)
              translated_features = transformation.reshape((num_sv, self.num_features))
              self.targ = np.append(self.targ, self.support_vector_targets)
              self.feat = np.vstack((translated_features, self.feat))


    def predict(self, X):

        vals = None
        ensemble_classes = None
        margin_total = None
        distance_mask = None

        for model in self.models:
            margins = model.decision_function(X)
            if margin_total is None:
                margin_total = margins
            else:
                margin_total = margin_total + margins
            classes = np.argmax(margins, axis=1)
            mask = np.zeros_like(margins)
            mask[np.arange(len(margins)), classes] = 1

            if ensemble_classes is None:
                ensemble_classes = classes.reshape(-1, 1)
            else:
                ensemble_classes = np.hstack((ensemble_classes, classes.reshape(-1, 1)))

            max_margins = margins[np.arange(len(classes)), classes]
            distances = max_margins * np.sum(np.square(margins - max_margins.reshape(-1, 1)),
                                             axis=1)
            if vals is None:
                vals = distances.reshape(-1, 1)
            else:
                vals = np.hstack((vals, distances.reshape(-1, 1)))

            mask = mask * distances.reshape(-1, 1)

            if distance_mask is None:
                distance_mask = mask
            else:
                distance_mask += mask
        # return mode(ensemble_classes, axis=1)[0] # 4.99
        # return np.argmax(margin_total, axis=1) # 4.89
        # return ensemble_classes[np.arange(len(ensemble_classes)), np.argmax(vals, axis=1)] # 4.79
        # return np.argmax(distance_mask, axis=1)
        return mode(ensemble_classes, axis=1)[0], np.argmax(margin_total, axis=1), ensemble_classes[
            np.arange(len(ensemble_classes)), np.argmax(vals, axis=1)], np.argmax(distance_mask,
                                                                                  axis=1)

    def error(self, X=None):
        if X is None:
            a, b, c, d = self.predict(self.test_features)
            print(np.round(100 - 100 * accuracy_score(self.test_targets, a), 2))
            print(np.round(100 - 100 * accuracy_score(self.test_targets, b), 2))
            print(np.round(100 - 100 * accuracy_score(self.test_targets, c), 2))
            print(np.round(100 - 100 * accuracy_score(self.test_targets, d), 2))

            # return 100
            # return np.round(100 - 100 * accuracy_score(self.test_targets, self.predict(self.test_features)), 2)
        else:
            return np.round(100 - 100 * accuracy_score(self.test_targets, self.predict(X)), 2)


In [0]:
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from mnist import MNIST

scaler = MinMaxScaler((-1, 1))

pd.set_option('display.width', 320)
np.set_printoptions(linewidth=320)

mndata = MNIST('/content')

train_features, train_targets = mndata.load_training()
test_features, test_targets = mndata.load_testing()
train_features = scaler.fit_transform(np.asarray(train_features))
test_features = scaler.fit_transform(np.asarray(test_features))
train_targets = np.asarray(train_targets)
test_targets = np.asarray(test_targets)



# num_f = 400
# train = pd.read_csv("/content/USPS_train.csv", delim_whitespace=True)
# test = pd.read_csv("/content/USPS_test.csv", delim_whitespace=True)

# train_features = train.iloc[:, 1:].to_numpy()
# train_targets = train.iloc[:, 0].to_numpy()
# test_features = test.iloc[:, 1:].to_numpy()
# test_targets = test.iloc[:, 0].to_numpy()

# train_features = np.pad(train_features.reshape((7290, 16, 16)), ((0, 0), (2, 2), (2, 2)),
#                         'constant',
#                         constant_values=(-1)).reshape((7290, num_f))
# test_features = np.pad(test_features.reshape((2006, 16, 16)), ((0, 0), (2, 2), (2, 2)), 'constant',
#                        constant_values=(-1)).reshape((2006, num_f))

print("Data has been read")

Data has been read


In [0]:

directions = [(1, 0),  # D
              (-1, 0),  # U
              (0, 1),  # R
              (0, -1),  # L
              (1, 1),  # RD
              (1, -1),  # LD
              (-1, 1),  # RU
              (-1, -1)]  # LU


start = timer()
# 5.28 is basic error C = 1
# 1.62 is basic error C = 10

print("Ensemble training, C = 1")
ensemble = EnsembleSVM(train_features, train_targets, test_features, test_targets, 1)
ensemble.add_rotation(-5, 5, 5)
ensemble.train(True)
ensemble.add_translations(directions[0:2], 1, 1)  # UD
ensemble.train()
ensemble.add_translations(directions[3:4], 1, 1)  # L
ensemble.add_translations(directions[5:6], 1, 1)  # LD
ensemble.add_translations(directions[7:8], 1, 1)  # LU
ensemble.train()
ensemble.add_translations(directions[2:3], 1, 1)  # R
ensemble.add_translations(directions[4:5], 1, 1)  # RD
ensemble.add_translations(directions[6:7], 1, 1)  # RU
ensemble.train()

end = timer()

print(f"Error: {ensemble.error()}")
print(f"Duration: {end - start} seconds")

start = timer()
print("Ensemble training, C = 10")
ensemble = EnsembleSVM(train_features, train_targets, test_features, test_targets, 10)
ensemble.add_rotation(-5, 5, 5)
ensemble.train(True)
ensemble.add_translations(directions[0:2], 1, 1)  # UD
ensemble.train()
ensemble.add_translations(directions[3:4], 1, 1)  # L
ensemble.add_translations(directions[5:6], 1, 1)  # LD
ensemble.add_translations(directions[7:8], 1, 1)  # LU
ensemble.train()
ensemble.add_translations(directions[2:3], 1, 1)  # R
ensemble.add_translations(directions[4:5], 1, 1)  # RD
ensemble.add_translations(directions[6:7], 1, 1)  # RU
ensemble.train()
end = timer()

print(f"Error: {ensemble.error()}")
print(f"Duration: {end - start} seconds")

Ensemble training, C = 10
0.9838
Support Vectors: 11702
Training set size: 35106
Training set size: 58464
Training set size: 77952
Training set size: 77952
1.01
0.99
1.03
1.0
Error: None
Duration: 6767.734113931001 seconds
